# **Description:**

The Weather Forecast Prediction project is focused on building a deep learning model using Long Short-Term Memory (LSTM) networks to predict weather conditions based on historical meteorological data. This project employs PyTorch as the deep learning framework, leveraging its flexibility and powerful tools to design, train, and evaluate the LSTM model. The goal is to predict future weather conditions by analyzing patterns in the past weather data, specifically focusing on four key features: mean temperature, humidity, wind speed, and atmospheric pressure.

# 1.Import Libraries

In [799]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim


# 2. Check for GPU

In [800]:
device='cuda:0' if torch.cuda.is_available() else 'cpu'


# 3. Import Dataset

This dataset was imported from kaggle website. It consists of many independent features like mean_temp,mean_pressure,wind speed, Humidity

# Reference: 
https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

In [801]:
dataset2=pd.read_csv('DailyDelhiClimateTrain.csv')
dataset2['date']=pd.to_datetime(dataset2['date'])

dataset=dataset2[['date','meantemp','humidity','wind_speed','meanpressure']].values


# 4. Split into Train and Validation Data

In [802]:
train_size=int(0.7*len(dataset))
train_data,val_data=dataset[:train_size],dataset[train_size:]
val_dates=dataset2['date'][train_size:]

Here we may no need of predicting dates which are sequential all over the dataset.So let's drop date column from dataset.

# 5. Normalization

In [803]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
scaled_train_data=sc.fit_transform(train_data[:,1:])
scaled_val_data=sc.transform(val_data[:,1:])

# 6. Create Inputs for model

In [804]:
def create_input(data,timestep=1):
    X,Y=[],[]
    for i in range(len(data)-timestep-1):
        X.append(scaled_train_data[i:(i+timestep)])
        Y.append(scaled_train_data[(i+timestep)])    
    return np.array(X),np.array(Y)


In [805]:
timestep=7
x_train,y_train=create_input(scaled_train_data,timestep)
x_val,y_val=create_input(scaled_val_data,timestep)

Here our dataset is in numpy form. To input them in to a pytorch model they must be in torch tensors. So let's change numpy inputs into tensors.

In [806]:
x_train = x_train.astype(np.float32)
x_val = x_val.astype(np.float32)
y_train=y_train.astype(np.float32)
y_val=y_val.astype(np.float32)

In [807]:
x_train=torch.tensor(x_train).float()
x_val=torch.tensor(x_val).float()
y_train=torch.tensor(y_train).float()
y_val=torch.tensor(y_val).float()


# 7. Build a Model
 Hence we are building a model to predict weather forecast which depends on their previous values we must use Recurrent neural Networks(RNN). But a simple RNN may not perform as good as well in a less number of epochs. So, let's choose LSTM.Let's also make our LSTM bidirectional so that while learning our model will learn features in both directions.We also implement dropout which helps in prevention of overfitting.

In [808]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        self.lstm=nn.LSTM(input_size=4,num_layers=2,hidden_size=128,batch_first=True,bidirectional=True)
        self.dropout=nn.Dropout(0.2)
        self.linear1=nn.Linear(128*2,64)
        self.linear2=nn.Linear(64,8)
        self.output_layer=nn.Linear(8,4)
    def forward(self,x):
        h0=torch.zeros(self.lstm.num_layers*2,x.size(0),self.lstm.hidden_size).to(device)
        c0=torch.zeros(self.lstm.num_layers*2,x.size(0),self.lstm.hidden_size).to(device)
        out,_=self.lstm(x,(h0,c0))
        out=self.dropout(out)
        out=self.linear1(out[:,-1,:])
        out=self.linear2(out)
        out=self.output_layer(out)
        return out



## Compile Model

Hence we are going to predict numerical values which comes under regression tasks MSELoss() will be suitable and Adam optimizer plays a crucial role in optimizing values for every epoch.

In [809]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
model=LSTM()
loss_fn=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
scheduler=ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=5,verbose=True)

c:\Users\tejat\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


## Fit Model

Let's fit our model for 20 epochs and also caluculate accuracy for every epoch for both training and validation data with a threshold range of 1.0. 

In [810]:
epochs=20
threshold=0.9
def inverse_transform(data, scaler):
    return sc.inverse_transform(data.reshape(1, -1)).flatten()
for epoch in range(1,epochs+1):
    model.train()
    y_logits=model(x_train).squeeze()
    y_pred=torch.round(torch.sigmoid(y_logits))
    #print(y_logits.shape)
    loss=loss_fn(y_logits,y_train)
    acc=torch.mean((torch.abs(y_train - y_pred) <= threshold).float(), dim=0)
    overall_accuracy = acc.mean().item() * 100  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    val_pred=model(x_val)
    val_loss=loss_fn(val_pred,y_val)
    acc_val=torch.mean((torch.abs(y_val - val_pred) <= threshold).float(), dim=0)
    overall_val_accuracy = acc_val.mean().item() * 100
    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Accuracy: {acc} | Overall Accuracy : {overall_accuracy} | Validation Loss: {val_loss:.5f} | Validation Accuracy :{acc_val} | Overall Validation Accuracy : {overall_val_accuracy}")

Epoch: 1 | Loss: 0.28206 | Accuracy: tensor([0.9970, 0.9921, 0.9990, 0.9901]) | Overall Accuracy : 99.45812821388245 | Validation Loss: 0.09214 | Validation Accuracy :tensor([1., 1., 1., 1.]) | Overall Validation Accuracy : 100.0
Epoch: 2 | Loss: 0.08540 | Accuracy: tensor([0.9970, 0.9921, 0.6956, 0.9901]) | Overall Accuracy : 91.8719232082367 | Validation Loss: 4.35315 | Validation Accuracy :tensor([0.0000, 0.0000, 0.0000, 0.1763]) | Overall Validation Accuracy : 4.408352822065353
Epoch: 3 | Loss: 4.34537 | Accuracy: tensor([0.9970, 0.9921, 0.6956, 0.9901]) | Overall Accuracy : 91.8719232082367 | Validation Loss: 0.05535 | Validation Accuracy :tensor([1., 1., 1., 1.]) | Overall Validation Accuracy : 100.0
Epoch: 4 | Loss: 0.06226 | Accuracy: tensor([0.9970, 0.9921, 0.6956, 0.9901]) | Overall Accuracy : 91.8719232082367 | Validation Loss: 0.23699 | Validation Accuracy :tensor([0.9884, 0.9907, 0.9954, 1.0000]) | Overall Validation Accuracy : 99.36195015907288
Epoch: 5 | Loss: 0.23129 | 

We have achieved a training accuracy of 91.87 and validation accuracy of 100%